In [36]:
from __future__ import print_function
import sys
from pyspark import SparkContext
from pyspark import SparkConf
from operator import add
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import StopWordsRemover
from gensim.models import KeyedVectors

In [37]:
spark = SparkSession\
        .builder\
        .appName("QuoraInsincere")\
        .getOrCreate()
# conf = SparkConf().setMaster("local").setAppName("sample")
# sc = SparkContext(conf=conf)
# conf = spark.conf
# sc = SparkContext(conf)
sc = spark.sparkContext

In [38]:
corpus = spark.read.option("header","true").option("inferSchema","true").csv("data/small.csv")

In [112]:
sentences = corpus.select('question_text').collect()

In [105]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in sentences:
        for word in sentence.split():
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [106]:
vocab = build_vocab(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

{'How': 190, 'did': 23, 'Quebec': 1, 'nationalists': 1, 'see': 6}


In [83]:
news_path = '/Users/ruolanzeng/InsincereQuestionClassification/data/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(news_path, binary=True)

In [93]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in vocab:
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [107]:
oov = check_coverage(vocab,embeddings_index)

Found embeddings for 94.87% of vocab
Found embeddings for  90.14% of all text


In [108]:
oov[:10]

[('to', 300),
 ('a', 296),
 ('of', 244),
 ('and', 209),
 ('2018', 13),
 ('2017', 7),
 ('100', 6),
 ('30', 5),
 ('20', 4),
 ('10', 3)]

In [113]:
def clean_text(x):

    
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

In [114]:
def build_vocab2(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in sentences: 
        for word in clean_text(sentence[0]).split():
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [115]:
vocab = build_vocab2(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

{'How': 190, 'did': 23, 'Quebec': 1, 'nationalists': 1, 'see': 6}


In [116]:
oov = check_coverage(vocab,embeddings_index)

Found embeddings for 94.87% of vocab
Found embeddings for  90.14% of all text


In [117]:
oov[:10]

[('to', 300),
 ('a', 296),
 ('of', 244),
 ('and', 209),
 ('2018', 13),
 ('2017', 7),
 ('100', 6),
 ('30', 5),
 ('20', 4),
 ('10', 3)]

In [118]:
import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [119]:
def build_vocab3(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in sentences: 
        for word in clean_numbers(clean_text(sentence[0])).split():
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [120]:
vocab = build_vocab3(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

{'How': 190, 'did': 23, 'Quebec': 1, 'nationalists': 1, 'see': 6}


In [121]:
oov = check_coverage(vocab,embeddings_index)

Found embeddings for 96.57% of vocab
Found embeddings for  90.94% of all text


In [122]:
oov[:20]

[('to', 300),
 ('a', 296),
 ('of', 244),
 ('and', 209),
 ('organisation', 2),
 ('favourite', 2),
 ('####…', 2),
 ('kardasian', 1),
 ('licencing', 1),
 ('diffferently', 1),
 ('Kubernetes', 1),
 ('kubernetes', 1),
 ('ask…', 1),
 ('didnt', 1),
 ('lifehacks', 1),
 ('Lyft', 1),
 ('Shouldnt', 1),
 ('wrastling', 1),
 ('maintanable', 1),
 ('ॡ', 1)]

In [123]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

In [124]:
to_remove = ['a','to','of','and']

def build_vocab4(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in sentences: 
        for word in replace_typical_misspell(clean_numbers(clean_text(sentence[0]))).split():
            if not word in to_remove:
                try:
                    vocab[word] += 1
                except KeyError:
                    vocab[word] = 1
    return vocab

In [125]:
vocab = build_vocab4(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

{'How': 190, 'did': 24, 'Quebec': 1, 'nationalists': 1, 'see': 6}
